# Online Inference with TabNet

In [6]:
import base64
import json
import pandas as pd
import pickle

from urllib3 import HTTPConnectionPool, HTTPSConnectionPool

## Calling our Model Endpoint

In [7]:
prediction_conn_pool = HTTPConnectionPool("localhost:5002")
prediction_headers = {"Content-Type": "application/json"}
def get_predictions(df):
    dfs = pickle.dumps(df)
    dfs = base64.b64encode(dfs).decode("utf-8")
    r = prediction_conn_pool.request(
        "POST",
        url="/predict/",
        headers=prediction_headers,
        body=json.dumps({"df": dfs}),
    )
    return json.loads(r.data.decode("utf-8"))

## Load some sample features

In [8]:
test_df = pickle.load(open('inference_df.p', 'rb'))
movie_ids = list(test_df.MOVIE_ID.values)
test_df.head(1)

,USER_ID,MOVIE_ID,MOVIE_GENRE_INFO__IS_ACTION,MOVIE_GENRE_INFO__IS_ADVENTURE,MOVIE_GENRE_INFO__IS_ANIMATION,MOVIE_GENRE_INFO__IS_CHILDREN,MOVIE_GENRE_INFO__IS_COMEDY,MOVIE_GENRE_INFO__IS_CRIME,MOVIE_GENRE_INFO__IS_DOCUMENTARY,MOVIE_GENRE_INFO__IS_DRAMA,...,USER_FANTASY_RATING_HISTORY__RATING_MEAN_730D_1D,USER_FILM_NOIR_RATING_HISTORY__RATING_MEAN_730D_1D,USER_HORROR_RATING_HISTORY__RATING_MEAN_730D_1D,USER_MUSICAL_RATING_HISTORY__RATING_MEAN_730D_1D,USER_MYSTERY_RATING_HISTORY__RATING_MEAN_730D_1D,USER_ROMANCE_RATING_HISTORY__RATING_MEAN_730D_1D,USER_SCI_FI_RATING_HISTORY__RATING_MEAN_730D_1D,USER_THRILLER_RATING_HISTORY__RATING_MEAN_730D_1D,USER_WAR_RATING_HISTORY__RATING_MEAN_730D_1D,USER_WESTERN_RATING_HISTORY__RATING_MEAN_730D_1D
0,102329,3114,0,1,1,1,1,0,0,0,...,3.714286,NaN,3.933333,4.0,3.538461,3.865854,3.875,3.904762,4.666667,NaN


## Test Inference

In [9]:
predictions = get_predictions(test_df)['predictions']

In [10]:
movieid2title = pickle.load(open("movieid2title.p", "rb"))
titles = [movieid2title[int(mid)] for mid in movie_ids]
pd.DataFrame({'Movie': titles, 'Predicted Score': predictions})

,Movie,Predicted Score
0,Toy Story 2 (1999),4.327136
1,Toy Story 3 (2010),4.430651
2,"Monsters, Inc. (2001)",4.461756
3,Finding Nemo (2003),4.293376
4,"Lion King, The (1994)",4.454514
5,"Incredibles, The (2004)",4.485878
6,Up (2009),4.474195
7,Shrek (2001),4.335835
8,Aladdin (1992),4.464612
9,Jurassic Park (1993),4.291188
